In [7]:
# -*- coding: utf-8 -*-

u"""
マルコフ連鎖を用いて適当な文章を自動生成するファイル
"""

import os.path
import sqlite3
import random

from prepare_saving_as_db import PrepareChain


class GenerateText(object):
    """
    文章生成用クラス
    """

    def __init__(self, n=100):
        """
        初期化メソッド
        @param n いくつの文章を生成するか
        """
        self.n = n

    def generate(self):
        """
        実際に生成する
        @return 生成された文章
        """
        # DBが存在しないときは例外をあげる
        if not os.path.exists(PrepareChain.DB_PATH):
            raise IOError("DBファイルが存在しません")

        # DBオープン
        con = sqlite3.connect(PrepareChain.DB_PATH)
        con.row_factory = sqlite3.Row

        # 最終的にできる文章
        generated_text = ""

        # 指定の数だけ作成する
        for i in range(self.n):
            text = self._generate_sentence(con)+'\n'+'\n'
            generated_text += text

        # DBクローズ
        con.close()

        return generated_text

    def _generate_sentence(self, con):
        """
        ランダムに一文を生成する
        @param con DBコネクション
        @return 生成された1つの文章
        """
        # 生成文章のリスト
        morphemes = []

        # はじまりを取得
        first_triplet = self._get_first_triplet(con)
        morphemes.append(first_triplet[1])
        morphemes.append(first_triplet[2])

        # 文章を紡いでいく
        while morphemes[-1] != PrepareChain.END:
            prefix1 = morphemes[-2]
            prefix2 = morphemes[-1]
            triplet = self._get_triplet(con, prefix1, prefix2)
            morphemes.append(triplet[2])

        # 連結
        result = "".join(morphemes[:-1])

        return result

    def _get_chain_from_DB(self, con, prefixes):
        u"""
        チェーンの情報をDBから取得する
        @param con DBコネクション
        @param prefixes チェーンを取得するprefixの条件 tupleかlist
        @return チェーンの情報の配列
        """
        # ベースとなるSQL
        sql = "select prefix1, prefix2, suffix, freq from chain_freqs where prefix1 = ?"

        # prefixが2つなら条件に加える
        if len(prefixes) == 2:
            sql += " and prefix2 = ?"

        # 結果
        result = []

        # DBから取得
        cursor = con.execute(sql, prefixes)
        for row in cursor:
            result.append(dict(row))

        return result

    def _get_first_triplet(self, con):
        """
        文章のはじまりの3つ組をランダムに取得する
        @param con DBコネクション
        @return 文章のはじまりの3つ組のタプル
        """
        # BEGINをprefix1としてチェーンを取得
        prefixes = (PrepareChain.BEGIN,)

        # チェーン情報を取得
        chains = self._get_chain_from_DB(con, prefixes)

        # 取得したチェーンから、確率的に1つ選ぶ
        triplet = self._get_probable_triplet(chains)

        return (triplet["prefix1"], triplet["prefix2"], triplet["suffix"])

    def _get_triplet(self, con, prefix1, prefix2):
        u"""
        prefix1とprefix2からsuffixをランダムに取得する
        @param con DBコネクション
        @param prefix1 1つ目のprefix
        @param prefix2 2つ目のprefix
        @return 3つ組のタプル
        """
        # BEGINをprefix1としてチェーンを取得
        prefixes = (prefix1, prefix2)

        # チェーン情報を取得
        chains = self._get_chain_from_DB(con, prefixes)

        # 取得したチェーンから、確率的に1つ選ぶ
        triplet = self._get_probable_triplet(chains)

        return (triplet["prefix1"], triplet["prefix2"], triplet["suffix"])

    def _get_probable_triplet(self, chains):
        u"""
        チェーンの配列の中から確率的に1つを返す
        @param chains チェーンの配列
        @return 確率的に選んだ3つ組
        """
        # 確率配列
        probability = []

        # 確率に合うように、インデックスを入れる
        for (index, chain) in enumerate(chains):
            for j in range(chain["freq"]):
                probability.append(index)

        # ランダムに1つを選ぶ
        chain_index = random.choice(probability)

        return chains[chain_index]


if __name__ == '__main__':
    generator = GenerateText()
    print(generator.generate())

なお、この問において「法」という

）に関する次の記述のうち、民法の規定に違反しないものはどれか

Ａは、専任の宅建取引士証」という

）に関する次の記述のうち、正しいものはどれか

遺言及び遺留分に関する次の記述のうち、民法の規定によれば、正しいものはどれか

宅地建物取引業者Ａが、B所有の甲土地の所有を目的として賃貸し、月額10万円（うち、誤っているものはどれか

）を、「意思表示をした

宅地建物取引業者Aが自ら売主として宅地建物取引業者AがBから住宅用地の購入の媒介を行うに際し、宅地建物取引業法（以下この問において「機構」という

宅地建物取引業者でない買主Ｂとの間で、平成15年9月1日に、Ｂ及びＣは、この問において「法」という

保証に関する次の記述のうち、正しいものはどれか

）が、Ｂがその土地上に、自分が居住して土地をＣに危害を加えた場合における次の記述のうち、正しいものはどれか

なお、この問において「区分建物」という

Ａには子Ｈがいる

次の記述のうち、民法の規定及び判例によれば、正しいものはどれか

宅地建物取引業保証協会（以下この問において「契約書面」と定めているものはどれか

なお、この問において「宅建取引士登録（以下この問において「保全措置をいう

宅地建物取引業法の規定によれば、誤っているものはどれか

独立行政法人住宅金融支援機構（以下この問において「免許」という

）に関する次の記述のうち、宅地建物取引業法の規定に違反するものはどれか

）の規定及び判例によれば、正しいものはどれか

宅地建物取引業法の規定及び判例によれば、正しいものはどれか

Ａとの間で締結した

宅地建物取引業者Ａは自己所有の甲不動産にそれぞれ案内所は甲契約及び乙契約の解除をする場合における当該媒介に係る登録免許税の課税に関する次の記述のうち、正しいものはいくつあるか

この場合、宅地建物取引業法の規定及び判例によれば、正しいものはどれか

次の記述のうち、正しいものはどれか

次の記述のうち、民法の規定及び判例によれば、誤っているものはどれか

）に関する次の記述のうち、最も不適当なものはどれか

次に掲げる開発行為（都市計画法第23条に規定する重要事項について説明を翌日に行うこととなって宅地建物取引業者でない買主Ｂと、適当なものはどれか

）の規定によれば正